In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split as tts

import warnings 
warnings.filterwarnings('ignore')

import ipywidgets as widgets
from IPython.display import display, clear_output


ModuleNotFoundError: No module named 'ipywidgets'

In [56]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
! jupyter serverextension enable voila --sys-prefix

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook notebook.exe run script server
troubleshoot trust

Jupyter command `jupyter-nbextension` not found.
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
       

In [6]:
file = open('image.jpg', "rb")
image = file.read()

image_headline = widgets.Image(value = image,
                              format = 'jpg',
                              width = '300')

vbox_headline = widgets.VBox([image_headline])

In [7]:
df = pd.read_csv('C:\\Users\\Asus\\Documents\\A_Data\\laptopData.csv')

In [8]:
df1 = df.dropna()

In [9]:
xres = []
yres = []
res = []
touch = []
for i in df1['ScreenResolution']:
    [x,y] = i.split(' ')[-1].split('x')
    if 'Touchscreen' in i:
        touch.append(1)
    else:
        touch.append(0)

    res.append(x+'x'+y)

df1['res'] = res
df1['touch'] = touch


In [10]:
df1['Inches'] = df1['Inches'].replace('?',float('0'))
df1['Inches'] = df1['Inches'].astype('float')


In [11]:
freq = []
proc = []

for i in df1['Cpu']:
    ent = ' '.join(i.split('GHz')[0].split(' ')[:3])
    freq.append(i.split('GHz')[0].split(' ')[-1])
    if 'Intel' in ent:
        if ent[6:] != 'Core i3' and ent[6:] != 'Core i5' and ent[6:] != 'Core i7':
            ent = 'Other Intel Core'
    else:
        ent = 'AMD Core'
    proc.append(ent)
       
df1['proc_freq'] = freq
df1['CPU_Brand'] = proc

In [12]:
ram = []
for i in df1['Ram']:
    ram.append(i[:-2])
    
df1['Ram'] = ram

In [13]:
storage = []
storage_type = []
dual_storage = []
dual_storage_type = []

df1 = df1[df1['Memory'] != '?']

for i in df1['Memory']:
    if '+' not in i:
        if 'TB' not in i.split(' ')[0]:
            storage.append(int(i.split(' ')[0].split('GB')[0]))
        else:
            storage.append(1000*int(float(i.split(' ')[0].split('TB')[0])))
                
        storage_type.append(' '.join(i.split(' ')[1:]))
        dual_storage.append(0)
        dual_storage_type.append(0)
    else:
        if 'TB' not in i.split('+')[0].split(' ')[0]:
            storage.append(int(i.split('+')[0].split(' ')[0].split('GB')[0]))
        else:
            storage.append(1000*int(float(i.split('+')[0].split(' ')[0].split('TB')[0])))
        storage_type.append(' '.join(i.split('+')[0].split(' ')[1:]))
        
        if 'TB' not in i.split('+')[1].split(' ')[2]:
            dual_storage.append(int(i.split('+')[1].split(' ')[2].split('GB')[0]))
        else:
            dual_storage.append(1000*int(float(i.split('+')[1].split(' ')[2].split('TB')[0])))
        dual_storage_type.append(' '.join(i.split('+')[1].split(' ')[3:]))

storage_type = list(map(lambda x: x.replace('HDD ', 'HDD'), storage_type))
storage_type = list(map(lambda x: x.replace('SSD ', 'SSD'), storage_type))
storage_type = list(map(lambda x: x.replace('Flash Storage ', 'Flash Storage'), storage_type))

       
df1['storage'] = storage
df1['storage_type'] = storage_type
df1['dual_storage'] = dual_storage
df1['dual_storage_type'] = dual_storage_type
df1['dual_storage_type'][df1['dual_storage_type'] == 0] = 'None' 

df1['total_storage'] = df1['storage'] + df1['dual_storage']

In [14]:
gpu = []

for i in df1['Gpu']:
    gpu.append(i.split(' ')[0])
    
df1['Gpu'] = gpu

In [15]:
weight =[]

for i in df1['Weight']:
    weight.append(i.split('kg')[0])

df1['Weight'] = weight
df1 = df1[df1['Weight'] != '?']
df1['Weight'] = df1['Weight'].astype('float64')
df1 = df1[df1['Weight'] >= 1]


In [16]:
def subcategory(text):
    if text =='Windows 10' or text =='Windows 7' or text == 'Windows 10 S ':
        return 'Windows'
    elif text =='macOS' or text == 'Mac OS X':
        return 'Mac'
    else:
        return 'Others'
df1['OpSys'] = df1['OpSys'].apply(lambda x:subcategory(x))

In [17]:
df1 = df1.drop(df1.columns[[4,5,7,18,20]],axis = 1)

In [18]:
st = []

for i in df1['storage']:
    if i == '1.0TB':
        st.append('1TB')
        
    else:
        st.append(i)
df1['storage'] = st

df1 = df1[df1['storage'] != '?']
df2 = df1.copy()

# Encoding

In [19]:
df2['Company'] = df1['Company'].astype('category').cat.codes
df2['TypeName'] = df1['TypeName'].astype('category').cat.codes
df2['Gpu'] = df1['Gpu'].astype('category').cat.codes
df2['res'] = df1['res'].astype('category').cat.codes
df2['touch'] = df1['touch'].astype('category').cat.codes
df2['OpSys'] = df1['OpSys'].astype('category').cat.codes
df2['CPU_Brand'] = df1['CPU_Brand'].astype('category').cat.codes
df2['storage_type'] = df1['storage_type'].astype('category').cat.codes
df2['dual_storage_type'] = df1['dual_storage_type'].astype('category').cat.codes

df2['Ram'] = df1['Ram'].astype('int32')
#df2['Weight'] = df1['Weight'].astype('float64')
df2['proc_freq'] = df1['proc_freq'].astype('float64')

# Company

In [20]:
#df2['Company'] = df1.Company.astype('category').cat.categories

dict1 = dict(enumerate(df1.Company.astype('category').cat.categories))
dict2 = {}

dict2 = dict([(v, [k for k, v1 in dict1.items() if v1 == v])
              for v in set(dict1.values())])
dict2 = {key: int(value[0]) for key, value in dict2.items()}


company_wid = widgets.Dropdown(
    options=list(dict2.items()),
    description='Brand:',
)

# Laptop Type

In [21]:
#df2['TypeName'] = df1.TypeName.astype('category').cat.categories
dict1 = dict(enumerate(df1.TypeName.astype('category').cat.categories))
dict2 = {}

dict2 = dict([(v, [k for k, v1 in dict1.items() if v1 == v])
              for v in set(dict1.values())])

dict2 = {key: int(value[0]) for key, value in dict2.items()}

lap_type_wid = widgets.Dropdown(
    options=list(dict2.items()),
    description='Type:',
)

# Screen Size

In [22]:
screen_size = list(np.sort(df1['Inches'].unique()))[1:]

screen_size_wid= widgets.Dropdown(
    options=screen_size,
    description='Screen Size:',
)

# RAM

In [23]:
ram = list(np.sort(df1['Ram'].unique().astype('int')))

ram_wid = widgets.Dropdown(
    options=ram,
    description='Ram:',
)

# GPU

In [24]:
#df2['Gpu'] = df1.Gpu.astype('category').cat.categories

dict1 = dict(enumerate(df1.Gpu.astype('category').cat.categories))
dict2 = {}

dict2 = dict([(v, [k for k, v1 in dict1.items() if v1 == v])
              for v in set(dict1.values())])
dict2 = {key: int(value[0]) for key, value in dict2.items()}

gpu_wid = widgets.Dropdown(
    options=list(dict2.items()),
    description='GPU:',
)

# Operating System

In [25]:

dict1 = dict(enumerate(df1.OpSys.astype('category').cat.categories))
dict2 = {}

dict2 = dict([(v, [k for k, v1 in dict1.items() if v1 == v])
              for v in set(dict1.values())])
dict2 = {key: int(value[0]) for key, value in dict2.items()}

op_sys_wid = widgets.ToggleButtons(
    options=list([('Windows', 2), ('Mac', 0), ('Others', 1)]),
    description='OS:',
    disabled=False,
    button_style='info',
     icons=['fa-brands fa-windows', 'fa-brands fa-apple', ' ']
)

# Weight

In [26]:

weight_wid = widgets.BoundedFloatText(
    min=0,
    max=9,
    step=0.1,
    description='Weight (kgs):',
    disabled=False
)

# Touchscreen

In [27]:

touch_wid = widgets.RadioButtons(
    options=[('Yes', 1), ('No', 0)],
    description='Touchscreen:',
    disabled=False
)

# Resolution

In [28]:
#df2['res'] = 
dict1 = dict(enumerate(df1.res.astype('category').cat.categories))
dict2 = {}

dict2 = dict([(v, [k for k, v1 in dict1.items() if v1 == v])
              for v in set(dict1.values())])
dict2 = {key: int(value[0]) for key, value in dict2.items()}

resol_wid = widgets.Dropdown(
    options=list(dict2.items()),
    description='Resolution:',
)

# CPU Frequency

In [29]:

freq_wid = widgets.FloatSlider(
    min=0.9,
    max=3.6,
    step=0.1,
    description='CPU Frequency:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)


# CPU Brand

In [30]:

dict1 = dict(enumerate(df1.CPU_Brand.astype('category').cat.categories))
dict2 = {}

dict2 = dict([(v, [k for k, v1 in dict1.items() if v1 == v])
              for v in set(dict1.values())])
dict2 = {key: int(value[0]) for key, value in dict2.items()}

cpu_wid = widgets.ToggleButtons(
    options=list(dict2.items()),
    description='CPU Brand:',
    disabled=False,
    button_style='info'
)

# Storage

In [31]:
store = list(np.sort(df1['storage'].unique())) # SelectionSlider

store_wid = widgets.Dropdown(
    options=store,
    description='Storage:',
)

# Storage Type

In [32]:

dict1 = dict(enumerate(df1.storage_type.astype('category').cat.categories))
dict2 = {}

dict2 = dict([(v, [k for k, v1 in dict1.items() if v1 == v])
              for v in set(dict1.values())])
dict2 = {key: int(value[0]) for key, value in dict2.items()}

store_type_wid = widgets.ToggleButtons(
    options=list(dict2.items()),
    description='Data Drive:',
    disabled=False,
    button_style='info'
)

#  Dual Storage

In [33]:

dict1 = dict(enumerate(df1.dual_storage_type.astype('category').cat.categories))
dict2 = {}

dict2 = dict([(v, [k for k, v1 in dict1.items() if v1 == v])
              for v in set(dict1.values())])
dict2 = {key: int(value[0]) for key, value in dict2.items()}

dual_store_wid = widgets.ToggleButtons(
    options=list(dict2.items()),
    description='Dual Storage',
    button_style='info'
)

In [34]:
x = df2[['Company', 'TypeName', 'Inches','Ram', 'Gpu', 'OpSys', 'Weight', 'res', 'touch', 
         'proc_freq', 'CPU_Brand', 'storage', 'storage_type', 'dual_storage_type']]
y=df2['Price']

In [35]:
x_train, x_test, y_train, y_test = tts(x,y, test_size= 0.2, random_state=101)

In [36]:
from sklearn.preprocessing import StandardScaler

sc=StandardScaler()
x_train_s=sc.fit_transform(x_train)
x_test_s=sc.transform(x_test)

In [37]:
model = RandomForestRegressor()
model.fit(x_train_s, y_train)

RandomForestRegressor()

In [38]:
#pred_rand = model.predict(x_test_s)

# Prediction Button

In [39]:
#from sklearn.metrics import accuracy_score, r2_score

#r2_score(y_test,pred_rand)

In [40]:
button_pred = widgets.Button(
                description='Predict the Price',
                style={'description_width': 'initial'}
            )


output = widgets.Output()

def on_button_clicked(event):
    with output:
        clear_output()
        print(f"The predicted value of the laptop you are looking for is ₹{np.round(model.predict([[company_wid.value, lap_type_wid.value, screen_size_wid.value, ram_wid.value, gpu_wid.value, op_sys_wid.value, weight_wid.value, resol_wid.value, touch_wid.value, freq_wid.value, cpu_wid.value, store_wid.value, store_type_wid.value, dual_store_wid.value]])[0],2)}")

button_pred.on_click(on_button_clicked)

vbox_result = widgets.VBox([button_pred, output])

In [41]:
vbox_text = widgets.VBox([company_wid,lap_type_wid, screen_size_wid, ram_wid, 
                          gpu_wid, op_sys_wid, weight_wid, resol_wid, touch_wid, freq_wid,
                          cpu_wid, store_wid, store_type_wid, dual_store_wid, vbox_result])


In [42]:
page = widgets.HBox([vbox_headline, vbox_text])
display(page)

In [49]:
! pip freeze > requiremen.txt

In [50]:
re = pd.read_csv('req_new.csv')

In [51]:
#re.head()

,req,Unnamed: 1
0,anyio @ file:///C:/b/abs_847uobe7ea/croot/anyi...,NaN
1,argon2-cffi @ file:///opt/conda/conda-bld/argo...,NaN
2,argon2-cffi-bindings @ file:///C:/ci/argon2-cf...,NaN
3,asttokens @ file:///opt/conda/conda-bld/asttok...,NaN
4,async-lru @ file:///C:/b/abs_e0hjkvwwb5/croot/...,NaN


In [52]:
new = []

for i in re['req']:
    new.append(i.split(' ')[0])
    
re['new'] = new
#re.head(50)

,req,Unnamed: 1,new
0,anyio @ file:///C:/b/abs_847uobe7ea/croot/anyi...,NaN,anyio
1,argon2-cffi @ file:///opt/conda/conda-bld/argo...,NaN,argon2-cffi
2,argon2-cffi-bindings @ file:///C:/ci/argon2-cf...,NaN,argon2-cffi-bindings
3,asttokens @ file:///opt/conda/conda-bld/asttok...,NaN,asttokens
4,async-lru @ file:///C:/b/abs_e0hjkvwwb5/croot/...,NaN,async-lru
5,attrs @ file:///C:/b/abs_35n0jusce8/croot/attr...,NaN,attrs
6,Babel @ file:///C:/b/abs_a2shv_3tqi/croot/babe...,NaN,Babel
7,beautifulsoup4 @ file:///C:/b/abs_0agyz1wsr4/c...,NaN,beautifulsoup4
8,bleach @ file:///opt/conda/conda-bld/bleach_16...,NaN,bleach
9,Bottleneck @ file:///C:/b/abs_f05kqh7yvj/croot...,NaN,Bottleneck


In [63]:
#new

['absl-py==0.12.0',
 'alabaster==0.7.12',
 'altair==4.1.0',
 'anaconda-client==1.7.2',
 'anaconda-navigator==1.10.0',
 'anaconda-project==0.8.3',
 'anyio==4.3.0',
 'argh==0.26.2',
 'argon2-cffi',
 'args==0.1.0',
 'asn1crypto',
 'astor==0.8.1',
 'astroid',
 'astropy==4.0.2',
 'astunparse==1.6.3',
 'async-generator==1.10',
 'atomicwrites==1.4.0',
 'attrs==23.2.0',
 'autopep8',
 'Babel==2.14.0',
 'backcall==0.2.0',
 'backports.functools-lru-cache',
 'backports.shutil-get-terminal-size==1.0.0',
 'backports.tempfile',
 'backports.weakref==1.0.post1',
 'backports.zoneinfo==0.2.1',
 'base58==2.1.0',
 'bcrypt',
 'beautifulsoup4',
 'bitarray',
 'bkcharts==0.2',
 'bleach',
 'blinker==1.4',
 'bokeh',
 'boto==2.49.0',
 'Bottleneck==1.3.2',
 'brewer2mpl==1.4.1',
 'brotlipy==0.7.0',
 'cachetools==4.2.1',
 'certifi==2020.12.5',
 'cffi',
 'chardet==4.0.0',
 'charset-normalizer==3.3.2',
 'click==7.1.2',
 'clint==0.5.1',
 'cloudpickle',
 'clyent==1.2.2',
 'cmake==3.18.4.post1',
 'colorama',
 'comtypes==

In [54]:
#csv_file_path = 'new_req_new_new.csv'

# Write the DataFrame to a new CSV file
#re.to_csv(csv_file_path, index=False)